In [13]:
import pandas as pd
import unicodedata
import re
from cleanco import basename
import abydos.distance as abd
from thefuzz import fuzz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import xlsxwriter
from tqdm import tqdm

In [6]:
# parameter to twik the sensitivity of the match,
# takes value between 0 to 1
# 1 means perfect match 
# 0 loose match
sesitivity_parameter=0.7

In [21]:
data=pd.read_csv("/Users/shashwatkumar/.mounty/Junkyard/compustat_glassdoor_match.csv")
data=data.dropna()
# test_data=data[0:10000]
# test_data.head()

In [31]:
def preprocess(company_name):
    company_name=company_name.lower()
    company_name=unicodedata.normalize('NFKD', company_name).encode('ASCII', 'ignore').decode()
    company_name=re.sub(r'[^\w\s]','',company_name)
    company_name=basename(company_name)
    # company_name=' '.join(re.sub(r'\b{}\b'.format(re.escape(suffix)), '', company_name).split())
    return company_name

def Discounted_Levenshtein(name1,name2):
    return abd.DiscountedLevenshtein().sim(name1,name2)

def str_subseq_ker_sim(name1,name2):
    return abd.SSK().sim(name1,name2)

def token_sort(name1,name2):
    return fuzz.token_sort_ratio(name1,name2)/100

def cosine_sim(name1,name2):
    try:
        count_vectorizer=CountVectorizer(ngram_range=(1, 1))
        # count_vectorizer = TfidfVectorizer(lowercase=False, analyzer="word", ngram_range=(1, 1))
        vector_matrix=count_vectorizer.fit_transform([name1,name2])
        # print(vector_matrix.todense())
        tokens=count_vectorizer.get_feature_names_out()
        # print(visualize(vector_matrix.toarray(),tokens))
        
        cosine_similarity_matrix = cosine_similarity(vector_matrix)
        # print(visualize(cosine_similarity_matrix,['name1','name2']))
        # print(cosine_similarity_matrix)
        return(cosine_similarity_matrix[0][1])
    except:
        return 0   
def visualize(matrix, tokens):

    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [32]:
def match_names(name1,name2):
    name1=preprocess(value[1])
    name2=preprocess(value[3])
    if (cosine_sim_score:=cosine_sim(name1,name2))<sesitivity_parameter:
        return None
    elif (dis_levevshtein_score:=Discounted_Levenshtein(name1,name2)) < sesitivity_parameter:
        return None
    elif (subseq_ker_score:=str_subseq_ker_sim(name1,name2))< sesitivity_parameter:
        return None
    elif (token_sort_score:=token_sort(name1,name2))<sesitivity_parameter:
        return None
    else:
        return [dis_levevshtein_score,subseq_ker_score, token_sort_score, cosine_sim_score]

In [33]:
columns=['id','comp_name','gvkey_code','comp_name1','similscore','Discounted_Levenshtein_score','subsequence_kernal_similarity_score','token_sort_score','cosine_similarity_score']
df=pd.DataFrame(columns=columns)
# writer = pd.ExcelWriter("output.xlsx")

In [34]:
for value in tqdm(data[963330:].values):
    print(value)
    score=match_names(name1=value[1],name2=value[3])
    if score != None:
        df.loc[len(df.index)]=np.append(value,score)

df.to_csv("output.csv")
# df.to_excel(writer)

  0%|          | 198/5318331 [00:00<1:52:20, 789.02it/s]

[3322 'am logic corporation' 1526 'c p l corp' 0.5091750772173156]
[58558 'sharp corporation' 1526 'c p l corp' 0.5217491947499509]
[37856 'leco corp' 1526 'c p l corp' 0.5270462766947299]
[16860 'cortec corporation' 1526 'c p l corp' 0.5333333333333333]
[16787 'corp2corp' 1526 'c p l corp' 0.5345224838248488]
[70696 'vicor corp.' 1526 'c p l corp' 0.5345224838248488]
[17743 'ct corp' 1526 'c p l corp' 0.5443310539518174]
[44333 'mv corp' 1526 'c p l corp' 0.5443310539518174]
[4454 'anvil corp' 1526 'c p l corp' 0.5555555555555556]
[10756 'c n x corp' 1526 'c p l corp' 0.5555555555555556]
[30010 'helac corp' 1526 'c p l corp' 0.5555555555555556]
[615 'abc corp.' 1526 'c p l corp' 0.5892556509887896]
[9111 'bmpc corp' 1526 'c p l corp' 0.5892556509887896]
[12284 'cbol corp' 1526 'c p l corp' 0.5892556509887896]
[43199 'mmc corp' 1526 'c p l corp' 0.629940788348712]
[4482 'ap corp' 1526 'c p l corp' 0.6804138174397717]
[2950 'allied mineral products company' 1528 'c p products corp' 0.5]

  0%|          | 1202/5318331 [00:00<41:44, 2123.36it/s] 

[1010 'acer america' 1543 'c v american corp' 0.5547001962252291]
[3596 'american foods' 1543 'c v american corp' 0.5547001962252291]
[3800 'american stone' 1543 'c v american corp' 0.5547001962252291]
[3826 'american trust' 1543 'c v american corp' 0.5547001962252291]
[3835 'american waste' 1543 'c v american corp' 0.5547001962252291]
[12068 'cash america' 1543 'c v american corp' 0.5547001962252291]
[18140 'daewon america' 1543 'c v american corp' 0.5547001962252291]
[21456 'eftec americas' 1543 'c v american corp' 0.5547001962252291]
[28969 'haeco americas' 1543 'c v american corp' 0.5547001962252291]
[35931 'keer america' 1543 'c v american corp' 0.5547001962252291]
[47186 'o & k american' 1543 'c v american corp' 0.5547001962252291]
[56155 'royal american' 1543 'c v american corp' 0.5547001962252291]
[58892 'sicl america' 1543 'c v american corp' 0.5547001962252291]
[70891 'virgin america' 1543 'c v american corp' 0.5547001962252291]
[74067 'yushin america' 1543 'c v american corp

  0%|          | 1429/5318331 [00:00<47:33, 1863.29it/s]

...

  0%|          | 21699/5318331 [00:08<31:41, 2785.07it/s]

[60215 'sonic systems international' 37427 'cai international inc'
 0.6587064830174596]
[2911 'alliant international university' 37427 'cai international inc'
 0.6594780076723941]
[26957 'geostabilization international' 37427 'cai international inc'
 0.6594780076723941]
[28063 'grant thornton international' 37427 'cai international inc'
 0.6594780076723941]
[40393 'market analytics international' 37427 'cai international inc'
 0.6594780076723941]
[43491 'money management international' 37427 'cai international inc'
 0.6594780076723941]
[48510 'orion international technologies' 37427 'cai international inc'
 0.6594780076723941]
[14974 'coastal international security' 37427 'cai international inc'
 0.6629935441317959]
[21704 'elg utica alloys international' 37427 'cai international inc'
 0.6629935441317959]
[29240 'harbisonwalker international' 37427 'cai international inc'
 0.6629935441317959]
[30097 'henderson international school' 37427 'cai international inc'
 0.6629935441317959]
[35

  0%|          | 22603/5318331 [00:08<34:03, 2590.94it/s]

[25387 'foundation for the national institutes of health' 37107
 'cal dive international inc' 0.5087230797227998]
[44864 'national telecommunications and information administration' 37107
 'cal dive international inc' 0.5088347164328333]
[19358 'development counsellors international' 37107
 'cal dive international inc' 0.5096471914376256]
[40230 'manufacturing resources international' 37107
 'cal dive international inc' 0.5096471914376256]
[33652 'international business sales and services' 37107
 'cal dive international inc' 0.5104812758612466]
[11034 'california national guard' 37107 'cal dive international inc'
 0.5111012519999519]
[12133 'catalina' 37107 'cal dive international inc' 0.5111012519999519]
[13836 'circle internet financial' 37107 'cal dive international inc'
 0.5111012519999519]
[21375 'educational testing service' 37107 'cal dive international inc'
 0.5111012519999519]
[44780 'national legal advisors' 37107 'cal dive international inc'
 0.5111012519999519]
[44847 'nati

KeyboardInterrupt: 